In [3]:
from pprint import pprint
from collections import namedtuple
import pickle

In [4]:
train_file="./data/ud-treebanks-conll2017/UD_English/en-ud-train.conllu"
test_file="./data/ud-treebanks-conll2017/UD_English/en-ud-dev.conllu"

In [5]:
word_info=namedtuple("word_info","id word pos head rel".split())

In [6]:
data=[]
sentence=[]

for line in open(train_file,"r",encoding="utf-8"):
    if not line.startswith("# "):
        line=line[:-1]
#         print(line)

        if len(line)!=0:
            ID,word,base, pos, pos1,_,head,rel,_,_=line.split("\t")
            if "." not in ID:
                ID=int(ID)-1
                head=int(head)-1

                sentence.append(word_info(ID,word,pos,head,rel))
        else:
            data.append(sentence)
            sentence=[]


In [26]:
pickle.dump(data,open("data.pkl","wb"))

In [24]:
data[0]

[word_info(id=0, word='Al', pos='PROPN', head=-1, rel='root'),
 word_info(id=1, word='-', pos='PUNCT', head=0, rel='punct'),
 word_info(id=2, word='Zaman', pos='PROPN', head=0, rel='flat'),
 word_info(id=3, word=':', pos='PUNCT', head=0, rel='punct'),
 word_info(id=4, word='American', pos='ADJ', head=5, rel='amod'),
 word_info(id=5, word='forces', pos='NOUN', head=6, rel='nsubj'),
 word_info(id=6, word='killed', pos='VERB', head=0, rel='parataxis'),
 word_info(id=7, word='Shaikh', pos='PROPN', head=6, rel='obj'),
 word_info(id=8, word='Abdullah', pos='PROPN', head=7, rel='flat'),
 word_info(id=9, word='al', pos='PROPN', head=7, rel='flat'),
 word_info(id=10, word='-', pos='PUNCT', head=7, rel='punct'),
 word_info(id=11, word='Ani', pos='PROPN', head=7, rel='flat'),
 word_info(id=12, word=',', pos='PUNCT', head=7, rel='punct'),
 word_info(id=13, word='the', pos='DET', head=14, rel='det'),
 word_info(id=14, word='preacher', pos='NOUN', head=7, rel='appos'),
 word_info(id=15, word='at', p

In [2]:
actions=[]
outputs=[]

for sent in data:
    stack=[word_info(-1,"ROOT","ROOT",None,None)]
    buffer=sent[:]
    while len(buffer)>0 and len(stack)>0:
       
        if len(stack)>2:
            next_child_id=next_child(stack[-1],buffer)
        
            while next_child_id!=0:
                stack,buffer,actions=swap_next_child(stack,buffer,actions,next_child_id)
                next_child_id=next_child(stack[-1],buffer)
                
            if stack[-1].head==stack[-2].id:
                _=stack.pop(-1)
                actions.append("right-reduce")
                print("{:15} [{:>20}] [{:40}]".format("right-reduce",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
            elif stack[-2].head==stack[-1].id:
                _=stack.pop(-2)
                actions.append("left-reduce")
                print("{:15} [{:>20}] [{:40}]".format("left-reduce",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
            else:
                stack.append(buffer.pop(0))
                actions.append("shift")
                print("{:15} [{:>20}] [{:40}]".format("shift",
                                                    " ".join([wi.word for wi in stack]),
                                                    " ".join([wi.word for wi in buffer])))
        elif len(stack)>1:
            next_child_id=next_child(stack[-1],buffer)
        
            while next_child_id!=0:
                stack,buffer,actions=swap_next_child(stack,buffer,actions,next_child_id)
                next_child_id=next_child(stack[-1],buffer)
                
            stack.append(buffer.pop(0))
            actions.append("shift")
            print("{:15} [{:>20}] [{:40}]".format("shift",
                                                    " ".join([wi.word for wi in stack]),
                                                    " ".join([wi.word for wi in buffer])))
            if stack[-1].head==stack[-2].id:
                _=stack.pop(-1)
                actions.append("right-reduce")
                print("{:15} [{:>20}] [{:40}]".format("right-reduce",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
            elif stack[-2].head==stack[-1].id:
                _=stack.pop(-2)
                actions.append("left-reduce")
                print("{:15} [{:>20}] [{:40}]".format("left-reduce",
                                            " ".join([wi.word for wi in stack]),
                                                " ".join([wi.word for wi in buffer])))
            
        else:
            stack.append(buffer.pop(0))
            actions.append("shift")
            print("{:15} [{:>20}] [{:40}]".format("shift",
                                                    " ".join([wi.word for wi in stack]),
                                                    " ".join([wi.word for wi in buffer])))
            if stack[-1].head==stack[-2].id:
                _=stack.pop(-1)
                actions.append("right-reduce")
                print("{:15} [{:>20}] [{:40}]".format("right-reduce",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
            elif stack[-2].head==stack[-1].id:
                _=stack.pop(-2)
                actions.append("left-reduce")
                print("{:15} [{:>20}] [{:40}]".format("left-reduce",
                                            " ".join([wi.word for wi in stack]),
                                                " ".join([wi.word for wi in buffer])))
    while len(buffer)==0 and len(stack)>2 and len(stack)>2:
        if stack[-1].head==stack[-2].id:
            _=stack.pop(-1)
            actions.append("right-reduce")
            print("{:15} [{:>20}] [{:40}]".format("right-reduce",
                                        " ".join([wi.word for wi in stack]),
                                        " ".join([wi.word for wi in buffer])))
        elif stack[-2].head==stack[-1].id:
            _=stack.pop(-2)
            actions.append("left-reduce")
            print("{:15} [{:>20}] [{:40}]".format("left-reduce",
                                        " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    
    outputs.append(actions)
    actions=[]

NameError: name 'data' is not defined

In [7]:
def swap_next_child(stack,buffer,actions,next_child_id):
    while stack[-1].id!=next_child_id:
        stack.append(buffer.pop(0))
        actions.append("shift")
        print("{:15} [{:>20}] [{:40}]".format("shift",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    while stack[-2].id!=stack[-1].head:
        buffer.insert(0,stack.pop(-2))
        actions.append("swap")
        print("{:15} [{:>20}] [{:40}]".format("swap",
                                            " ".join([wi.word for wi in stack]),
                                            " ".join([wi.word for wi in buffer])))
    return stack,buffer,actions

In [8]:
def next_child(word,buffer):
    for wi in buffer:
        if wi.head==word.id:
            return wi.id
    return 0

In [11]:
pickle.dump(outputs,open("parsing_sequences.pkl","wb"))